In [16]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep = ',')
melb_df = melb_data.copy()
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)


In [17]:
# давайте создадим переменную total_rooms, в которой будем хранить общее количество комнат в здании. 
# Для этого выполним сложение столбцов с количеством комнат, ванн и спален:
total_rooms = melb_df['Rooms'] + melb_df['Bathroom'] + melb_df['Bedroom']
display(total_rooms)

0         5
1         5
2         8
3         8
4         8
         ..
13575    10
13576     8
13577     8
13578     9
13579     9
Length: 13580, dtype: int64

In [18]:
#А теперь введём признак MeanRoomsSquare, который соответствует средней площади одной комнаты для каждого объекта.
#Для этого разделим площадь здания на полученное ранее общее количество комнат:
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsSquare'])

0        25.200000
1        15.800000
2        18.750000
3        15.750000
4        17.750000
           ...    
13575    12.600000
13576    16.625000
13577    15.750000
13578    17.444444
13579    12.444444
Name: MeanRoomsSquare, Length: 13580, dtype: float64

In [19]:
#Можно ввести ещё один интересный признак — AreaRatio, 
# коэффициент соотношения площади здания (BuildingArea) и площади участка (Landsize). 
# Для этого разницу двух площадей поделим на их сумму:
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area / sum_area
display(melb_df['AreaRatio'])

#Что показывает такой коэффициент? Если присмотреться, можно увидеть, что AreaRatio лежит в интервале от -1 до 1.
# Рассмотрим три случая, чтобы понять его значение:

#Если рассматриваемые площади равны, то числитель дроби зануляется и коэффициент тоже равен 0.
#Если одна из площадей начинает доминировать над другой, то коэффициент начинает расти в отрицательную сторону, если площадь участка больше площади здания, и в положительную сторону, если наоборот.
#Наконец, в предельном случае, если площадь здания равна 0, то числитель дроби равен знаменателю со знаком минус, а коэффициент равен -1, а если площадь участка равна 0, то числитель дроби равен знаменателю со знаком плюс, а коэффициент равен 1.

# Таким образом, значение в столбце AreaRatio служит своеобразным указателем соотношения площадей объекта недвижимости. Для пустырей — участков без строений — он будет равен -1, для домов без территории — 1, во всех остальных случаях мы можем увидеть, какая площадь больше — здания или участка.

0       -0.231707
1       -0.327660
2        0.056338
3        0.145455
4        0.083969
           ...   
13575   -0.676093
13576   -0.429185
13577   -0.551601
13578   -0.693060
13579   -0.527426
Name: AreaRatio, Length: 13580, dtype: float64

In [20]:
#Напишите функцию delete_columns(df, col=[]), которая удаляет столбцы из DataFrame и возвращает новую таблицу. 
#Если одного из указанных столбцов не существует в таблице, то функция должна возвращать None. 

customer_df = pd.DataFrame({
            'number': [0, 1, 2, 3, 4],
            'cust_id': [128, 1201, 9832, 4392, 7472],
            'cust_age': [13, 21, 19, 21, 60],
            'cust_sale': [0, 0, 0.2, 0.15, 0.3],
            'cust_year_birth': [2008, 2000, 2002, 2000, 1961],
            'cust_order': [1400, 14142, 900, 1240, 8430]
        })

def delete_columns(df, col=[]):
    for cc in col:
        if cc not in df.columns:
            return None
    return df.drop(col, axis=1)

    

In [21]:
#Задан DataFrame countries_df, содержащий следующие столбцы: название страны, население (population) 
# в миллионах человек и площадь страны (square) в квадратных километрах.
#Для каждой страны рассчитайте плотность населения (количество человек на квадратный километр).
#Затем по полученным данным рассчитайте среднее по плотностям населения в указанных странах. Ответ округлите до сотых.
#Плотность населения рассчитывается как количество человек, проживающих на территории отдельной страны,
# делённое на площадь этой страны. Обратите внимание, что население в таблице представлено в миллионах.

countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
countries_df['density'] = countries_df['population'] / countries_df['square'] * 1e6
print(round(countries_df['density'].mean(), 2))


84.93


In [22]:
display(melb_df['Date'])

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

In [23]:
#Для того чтобы преобразовывать столбцы с датами, 
# записанными в распространённых форматах, в формат datetime, 
# можно воспользоваться функцией pandas.to_datetime().
# В нашем случае в функции нужно указать параметр dayfirst=True, 
# который будет обозначать, что в первоначальном признаке первым идет день. 
# Преобразуем столбец Date в формат datetime, передав его в эту функцию:

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

In [24]:
year_sold = melb_df['Date'].dt.year
print(year_sold)

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int64


In [25]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: MonthSale, dtype: float64

In [26]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01')
display(delta_days)
display(delta_days.dt.days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

In [27]:
#Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. 
# Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. 
# Результат оформим в виде столбца AgeBuilding:

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df["AgeBuilding"])
melb_df = melb_df.drop('YearBuilt', axis=1)

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

In [28]:
display(melb_df.head().T)

,0,1,2,3,4
Suburb,Abbotsford,Abbotsford,Abbotsford,Abbotsford,Abbotsford
Address,85 Turner St,25 Bloomburg St,5 Charles St,40 Federation La,55a Park St
Rooms,2,2,3,3,4
Type,h,h,h,h,h
Price,1480000.0,1035000.0,1465000.0,850000.0,1600000.0
Method,S,S,SP,PI,VB
SellerG,Biggin,Biggin,Biggin,Biggin,Nelson
Date,2016-12-03 00:00:00,2016-02-04 00:00:00,2017-03-04 00:00:00,2017-03-04 00:00:00,2016-06-04 00:00:00
Distance,2.5,2.5,2.5,2.5,2.5
Postcode,3067,3067,3067,3067,3067


In [29]:
#Создайте в таблице melb_df признак WeekdaySale (день недели). 
# Найдите, сколько объектов недвижимости было продано в выходные (суббота и воскресенье), 
# результат занесите в переменную weekend_count. 
# В качестве ответа введите результат вывода переменной weekend_count.

melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
print(weekend_count)

12822


In [30]:
#Вам представлены данные (в формате csv) об отчётах очевидцев НЛО в США за период с 1930 по 2020 год.

# В данных есть следующие признаки:

#         "City" — город, где был замечен НЛО;
#         "Colors Reported" — цвет объекта;
#         "Shape Reported" — форма объекта;
#         "State" — обозначение штата;
#         "Time" — время, когда был замечен НЛО (данные отсортированы от старых наблюдений к новым). 

# Прочитайте данные, сделайте преобразование времени к формату datetime и выполните задания ниже.

import pandas as pd
ufo_df = pd.read_csv('data/ufo.csv', sep=',')
ufo_df['Time'] = pd.to_datetime(ufo_df['Time'])


# ufo_df['Years_View'] = ufo_df['Time'].dt.year
# display(ufo_df['Years_View'].mode()[0])

#Подсказка (1 из 1): Для удобства можно создать дополнительный столбец с датой Date,
# выделив атрибут datetime. После этого необходимо произвести фильтрацию и отобрать из данных только штат Невада.
# В отфильтрованных данных нужно вычислить разницу между двумя последовательными наблюдениями с помощью метода diff()
# и посчитать среднее количество дней между двумя наблюдениями.
ufo_df['Date'] = ufo_df['Time'].dt.date
display(ufo_df[ufo_df['State'] == 'NV']['Date'].diff().dt.days.mean())

68.92932862190813

In [32]:
melb_df.head()
display(melb_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Address          13580 non-null  object        
 2   Rooms            13580 non-null  int64         
 3   Type             13580 non-null  object        
 4   Price            13580 non-null  float64       
 5   Method           13580 non-null  object        
 6   SellerG          13580 non-null  object        
 7   Date             13580 non-null  datetime64[ns]
 8   Distance         13580 non-null  float64       
 9   Postcode         13580 non-null  int64         
 10  Bedroom          13580 non-null  int64         
 11  Bathroom         13580 non-null  int64         
 12  Car              13580 non-null  int64         
 13  Landsize         13580 non-null  float64       
 14  BuildingArea     13580 non-null  float

None